# KniT Anemia Route Optimization

 ## Objective
### a tool that can be used by survey supervisors to optimize the surveying strategy in terms of scheduling village visits and choosing sites for base of operations. 

### The tool will aid in choosing an optimal survey schedule for the next day, and can be rerun at the end of each day to update the schedule with latest data.

## Algorithm

### Mixed linear programming has been used to solve the problem. 

### The engine uses Google OR tools, an opensource, free to use library for commercial and research purposes. 

### The routing library of the Google OR has been used to formulate the problem so as to ensure that the same engine can be easily modified to include other features. https://developers.google.com/reference/constraint_solver/routing/.

## Version Specification

### This version invoves the routing when there are no time constraints associated with the villages

### This Version requires 2 files in the same directory for data:   Travel_times.csv and Demands.csv. 

### Demands.csv has the time requirement of every village encoded in seconds and the column that has the requirement has to specified in the program.  The programe takes by default the column 5 to contain the demands. 

### Travel_times.csv contains a matrix in the csv format that represent the travel times required to travel from one village to another. Its defined as d[i][j] = time taken to travel from village i to village j.

# Python code 


## header files

importing required libraries for output processing

In [ ]:
from __future__ import print_function
from six.moves import xrange

set the path to required directory in your system 

In [ ]:
import sys
sys.path.append("F:\\projects\\ameen\\ortools")

import the google or-tools required to solve the problem

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

import the csv library for input handling

In [ ]:
import csv

## Problem Data Definition

### class definition - Vehicle()
this class is associated with the properties of one particular vehicle
the capacity , which is the maximum time(in seconds) that a vehicle spends outside the central facility is given as an input by the user
#### input - max time that can be spend outside the central facility in seconds

In [ ]:
class Vehicle():
    """Stores the property of a vehicle"""
    def __init__(self):
        """Initializes the vehicle properties"""
        vehicle_capacity = input()
        self._capacity = int(vehicle_capacity)


    @property
    def capacity(self):
        """Gets vehicle capacity"""
        return self._capacity


 ### class definition - DataProblem()
 
 #### This class handles all the data associated with the problem 
#### Any change in input methods or formats will have to be reflected in the methods of this class

In [ ]:
class DataProblem():
    """Stores the data for the problem"""
    def __init__(self):
        """Initializes the data for the problem"""
        self._vehicle = Vehicle()
        #self._num_vehicles = 2

        self._depot = 3

        self._req_vehicles = 0

        #read the demands as a list of lists
        with open('test_demands1.csv', 'rU') as f:
            list = [i[5] for i in csv.reader(f)]
        list.pop(0)
        self._demands = [int(i) for i in list]

        with open('travel_times.csv', 'rU') as f:
            list = [i for i in csv.reader(f)]
        list.pop(0)
        #loop for the size of list
        self._distances = [[int(j) for j in i] for i in list]
         

        
    @property        
    def vehicle(self):
        """Gets a vehicle"""
        return self._vehicle

    @property
    def demands(self):
        """Gets demands at each location"""
        return self._demands

    @property
    def num_locations(self):
        """Gets number of locations"""
        return len(self.demands)

    @property
    def num_vehicles(self):
        """Gets number of vehicles"""
        return len(self.demands)-1

    @property
    def req_vehicles(self):
        """Gets number of vehicles"""
        return self._req_vehicles

    @property       
    def depot(self):
        """Gets depot location index"""
        return self._depot

    @property
    def distances(self):
        """Gets distance between each pair of locations"""
        return self._distances

    def printDemands(self):
        for i in range(len(self._demands)): 
            print('{0} , '.format(self._demands[i]))

    """the formulator is to visit the villages which has more demands than that compleatable in a single visit"""
    """that falls outside the scope of optimizationa and should be visited before optimization - so as to reduce the demands to what is compleatable by a single visit"""
    def formulator(self):
        print('formulator\n')

        for x in xrange(0, len(self._demands)):
            if ((self._demands[x] + 2*self._distances[self._depot][x]) > self._vehicle.capacity):
                while ((self._demands[x] + 2*self._distances[self._depot][x]) > self._vehicle.capacity) :
                    self._req_vehicles += 1
                    """updating the demands as we visit the villages"""
                    self._demands[x] -= (self._vehicle.capacity - 2*self._distances[self._depot][x])
                    print('Route of Vehilce {}\n'.format(self._req_vehicles))
                    print('{0} Load({1}) -> {2} Load({3}) -> {4} Load({5})\n'.format(self._depot, 0,  x, self._vehicle.capacity - 2*self._distances[self._depot][x], self._depot, self._vehicle.capacity))

##  Problem Constraints

### class definition - CreateDistanceEvaluator()
#### this class has methods related to the distance(will be stored in terms of time required to travel , in seconds) betwen any pair of villages 
#### the name distance can be misleading - it represets the time required in seconds

In [ ]:
class CreateDistanceEvaluator(object): # pylint: disable=too-few-public-methods
    """Creates callback to return distance between points."""
    def __init__(self, data):
        """Initializes the distance matrix."""
        self._distances = data.distances

    def distance_evaluator(self, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return self._distances[from_node][to_node]

### class definition - CreateDemandEvaluator()
#### this class has methods related to the demands of each village
#### demand of a village - time in seconds to be spend at that village to compleate the surey process

In [ ]:
class CreateDemandEvaluator(object): # pylint: disable=too-few-public-methods
    """Creates callback to get demands at each location."""
    def __init__(self, data):
        """Initializes the demand array."""
        self._demands = data.demands

    def demand_evaluator(self, from_node, to_node):
        """Returns the demand of the current node"""
        del to_node
        return self._demands[from_node]


### class definition - CreateTimeEvaluator()
#### this class has methods related to the totla travel time from one village to another 
#### compleating the survey in a village plus moving to the next village 

In [ ]:
class CreateTimeEvaluator(object):
    """Creates callback to get total times between locations."""
    @staticmethod
    def service_time(data, node):
        """Gets the service time for the specified location."""
        return data.demands[node]

    @staticmethod
    def travel_time(data, from_node, to_node):
        """Gets the travel times between two locations."""
        if from_node == to_node:
            travel_time = 0
        else:
            travel_time = data.distances[from_node][to_node]
        return travel_time

    def __init__(self, data):
        """Initializes the total time matrix."""
        self._total_time = {}
        # precompute total time to have time callback in O(1)
        for from_node in xrange(data.num_locations):
            self._total_time[from_node] = {}
            for to_node in xrange(data.num_locations):
                if from_node == to_node:
                    self._total_time[from_node][to_node] = 0
                else:
                    self._total_time[from_node][to_node] = int(
                        self.service_time(data, from_node) +
                        self.travel_time(data, from_node, to_node))

    def time_evaluator(self, from_node, to_node):
        """Returns the total time between the two nodes"""
        return self._total_time[from_node][to_node]

### function - add capacity constrain to the problem
#### The maximum time spent by each group of interviewers outside the central facility (maximum working hour limit) < Q


In [ ]:
def add_capacity_constraints(routing, data, time_evaluator):
    """Adds capacity constraint"""
    capacity = "Capacity"
    routing.AddDimension(
        time_evaluator,
        0, # null capacity slack
        data.vehicle.capacity, # vehicle maximum capacity
        True, # start cumul to zero
        capacity)